In [0]:
!pip install tweet-preprocessor
!pip install google-api-python-client

import pandas as pd
import preprocessor as p
import networkx as nx
import matplotlib.pyplot as plt
import tweepy
import json
import json
import requests
import csv
import re
import sys
import urllib.parse

from sklearn.feature_extraction.text import CountVectorizer

from googleapiclient.discovery import build
from datetime import datetime

In [0]:
!pip install nltk
import nltk 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [0]:
timeline_data = {}
news_api_title = []
news_api_decription = []
news_api_date = []
news_api_url = []

def decode_title(dct):
    if "title" in dct:
        news_api_title.append(p.clean(dct["title"]))
    else:
        return [""]
def decode_description(dct):
    if "description" in dct:
        news_api_decription.append(p.clean(dct["description"]))
    else:
        return [""]
def decode_date(dct):
    if "publishedAt" in dct:
        news_api_date.append(dct["publishedAt"])
    else:
        return [""]
def decode_url(dct):
    if "url" in dct:
        news_api_url.append(dct["url"])
    else:
        return [""]

In [0]:
def getTimelineData():
    for i in range(len(news_api_title)):
      if news_api_date[i][:10] not in timeline_data:
        timeline_data[news_api_date[i][:10]] = []
        temp=[]
        temp.append(news_api_title[i])
        temp.append(news_api_url[i])
        timeline_data[news_api_date[i][:10]].append(temp)
      else:
        temp=[]
        temp.append(news_api_title[i])
        temp.append(news_api_url[i])
        timeline_data[news_api_date[i][:10]].append(temp)

In [0]:
def getNewsAPI(query):
  keywords = urllib.parse.quote(query)
  url = "https://newsapi.org/v2/everything?q="+keywords+"&from=&apiKey=2ed8bf16847a458d9af4ed36ce4f979b"
  response = requests.get(url)
  newsApi_json = json.dumps(response.json(),sort_keys=True)
  # print(newsApi_json)
  # Decode title
  parsed = json.loads(newsApi_json,object_hook=decode_title)
  # Decode description
  parsed = json.loads(newsApi_json,object_hook=decode_description)
  # Decode date
  parsed = json.loads(newsApi_json,object_hook=decode_date)
  # Decode url
  parsed = json.loads(newsApi_json,object_hook=decode_url)
  getTimelineData()
getNewsAPI("kobe")
print(timeline_data)
print(len(timeline_data))
print(len(news_api_title))
print(len(news_api_url))
# print(len(news_api_title))
# print(len(news_api_url))
news_api_title = []
news_api_decription = []
news_api_date = []
news_api_url = []
timeline_data = {}



In [0]:
def getGNewsAPI(query):
  initialize_corpus()
  keywords = urllib.parse.quote(query)
  url = "https://gnews.io/api/v3/search?q="+keywords+"&token=a6419076f909fc74c42016c6bebf0755"
  response = requests.get(url)
  gNewsApi_json = json.dumps(response.json(), indent = 4,sort_keys=True)
  print(gNewsApi_json)
  # Decode title
  parsed = json.loads(gNewsApi_json,object_hook=decode_title)
  # Decode description
  parsed = json.loads(gNewsApi_json,object_hook=decode_description)
  # Decode date
  parsed = json.loads(gNewsApi_json,object_hook=decode_date)
  # Decode url
  parsed = json.loads(gNewsApi_json,object_hook=decode_url)
  getTimelineData()


In [0]:
def buildGraph():
  timeline_data = {}
  keyword = "Kobe"
  getNewsAPI(keyword)
  getGNewsAPI(keyword)
  print(timeline_data)

In [0]:
import requests
from bs4 import BeautifulSoup
def getArticle(url):
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)

    article_text = ''
    article = soup.findAll('p')
    for element in article:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text